In [27]:
import keras
import math
import pandas as pd
import numpy as np
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
import time
from torch.utils.data import Dataset, DataLoader
import pickle
pd.set_option('display.max_rows', 500)
import os
import tensorflow as tf
import torch
import torch.nn as nn
from math import sqrt
# import rmse from sklearn
from sklearn.metrics import mean_squared_error


# define random seeds for Neural Networks
torch.manual_seed(0)
np.random.seed(0)
tf.random.set_seed(0)
# ignore warnings jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# OWRI FRAMEWORK

In [86]:
def merge_trejectory_data(results, trajectory, direction):
    data = pd.DataFrame()
    for intersection_name in results[trajectory][direction]['raw']:
        intersection = results[trajectory][direction]['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [29]:
def merge_trejectory_data_metr(results):
    data = pd.DataFrame()
    for intersection_name in results['raw']:
        intersection = results['raw'][intersection_name]
        intersection = intersection.rename(columns={"cars": intersection_name})
        intersection = intersection.set_index(pd.DatetimeIndex(intersection['timestamp']))
        intersection = intersection.drop(columns=['timestamp'])
        data = pd.merge(data, intersection, left_index=True, right_index=True, how='outer')
    data.dropna(inplace=True)
    return data

In [30]:
def preprocess_df(df,n_obs, n_features, sequence_length):
    #do scaling:
    scaler = StandardScaler()
    df_train = df.values
    train_X, train_y = df_train[:, :n_obs], df_train[:, -n_features]
    scl = scaler.fit(train_X) # fit only on training data
    train_X = scl.transform(train_X)
    train_X = train_X.reshape((train_X.shape[0], sequence_length, n_features))
    return train_X, train_y, scl

In [31]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [32]:
class LSTM_uni(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device = 'mps',layer_dim=1, dropout_prob = 0.2):
        super(LSTM_uni, self).__init__()
        self.hidden_dim = hidden_dim # number of hidden units in hidden state
        self.layer_dim = layer_dim # number of stacked lstm layers
        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_dim, output_dim) # fully connected layer

    def forward(self, x, future=False):
        # input x is expected to be of shape (batch_dim, seq_dim, feature_dim)
        # hidden and cell states are expected along with input x in LSTMs = (h_0, c_0)
        # Initialize hidden state with zeros (layer_dim, batch_size, hidden_dim)
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim, device=device).requires_grad_()

        # LSTM output is Outputs: output, (h_n, c_n)
        # output is of shape (batch_dim, seq_dim, hidden_dim), h_n and c_n are of shape (layer_dim, batch_dim, hidden_dim)
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))
        out = out[:, -1, :] # only take the last output of the sequence
        out = self.fc(out) # fully connected layer

        return out

In [33]:
def train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=100):
    history = {}
    history['train_loss'] = []

    train_X_loader = DataLoader(train_X, batch_size=128, shuffle=False)
    train_y_loader = DataLoader(train_y, batch_size=128, shuffle=False)

    for epoch in range(epochs):
        history[epoch] = []
        ep_start = time.time()
        running_loss = 0.0
        for bx, data in enumerate(zip(train_X_loader,train_y_loader)):
            X = data[0].to(device)
            y = data[1].to(device)
            bt = model(X)
            loss = loss_fn(bt.reshape(-1), y.reshape(-1)) # calculate loss for input and recreated output
            history[epoch].append(loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            running_loss += loss.item()
        epoch_loss = running_loss/train_X.shape[0]
        history['train_loss'].append(epoch_loss)

    return history, model
    

In [34]:
def model_evaluation( model, test_X, device):
    test_X_loader = DataLoader(test_X, batch_size=128, shuffle=False)
    model = model.eval()
    preds = []
    with torch.no_grad():
        for bx, data in enumerate(test_X_loader):
            X = data.to(device)
            bt = model(X)
            preds.append(bt.cpu().numpy())
    preds = np.vstack(preds)
    preds = preds.reshape(-1)
    return preds

In [35]:
def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    return model, optimizer

In [58]:
def test_then_train_weighted_model(data, ae_score, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain,fixed_base_corr_df, pre_trained, static , base ):
    result_dict = {}

    # ------------------------------------ data processing ---------------------------------------- #
    if static == False:
        top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
    else:
        top_corr_df = fixed_base_corr_df
    isct_inc = top_corr_df.index.tolist()
    print("correlated intersections: {}".format(isct_inc))
    df = data[isct_inc].copy(deep=True)
    df = df.mul(top_corr_df, axis=1)
    df = df.astype('float32')
    n_features = len(isct_inc) # number of features (correlated intersections)
    n_obs = sequence_length * n_features # number of columns in the input
    reframed = series_to_supervised(df, sequence_length, output_pred)
    train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


    # # # ------------------------------------ modelling ---------------------------------------------- #
    # define model, loss function and optimizer
    model = LSTM_uni(input_dim = n_features, hidden_dim = hidden_size, output_dim = output_pred, layer_dim = num_layers, dropout_prob= dropout, device = device)
    model = model.to(device)
    loss_fn = torch.nn.MSELoss()
    optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
    if pre_trained==True:
        model, optimiser = load_ckp(results_load_path, model, optimiser)
    

    # ------------------------------------ testing ---------------------------------------------- #
    yhat = model_evaluation( model, train_X , device)
    result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
    result_dict['MAE'] = mean_absolute_error(yhat,train_y)
    result_dict['R2'] = r2_score(yhat,train_y)
    result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
    print("RMSE: {}".format(result_dict['RMSE']))


   # ------------------------------------ training ---------------------------------------------- #
    if not base:
        start = time.time()
        if pre_trained==False:
            history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=epoch_train)
        else:
            history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=epoch_retrain)
        end = time.time()
        checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
        torch.save(checkpoint, results_save_path)
        print("Training time: {}".format(end-start))
        result_dict['history'] = history
        result_dict['intersedctions'] = isct_inc
        result_dict['train_time'] = end-start

    
    return result_dict, top_corr_df

#### ---------------------------- Hauge data processing ---------------------------- 

In [81]:
# results save path
outlier_model_name = 'PW-AE'
data_name = 'hauge'
base_result_path = f'../results/{data_name}/real_time_modeling'

In [82]:
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [83]:
# load data of correlated results from pickle file
with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [84]:
thresholds = [0,0.05,0.1,0.25,0.5, 1] # thresholds for the percentage of correlated columns to keep
target_intersections={"T1":{"North":"K504", "South":"K561"}, # get target intersections for each trajectory and direction
                      "T2":{"North":"K703", "South":"K206"}}
# declare variables
threshold = 0.05
time_window = 3
epoch_train = 100
epoch_retrain = 20
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
base_train_portion = 0.5 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [90]:
errors={}
for trajectory in results.keys():
    errors[trajectory]={}
    for direction in results[trajectory]:
        print(f"\n \n ------------------------ Starting trajectory {trajectory}, direction {direction} and threshold: {threshold} ------------------------")
        key = trajectory+"_"+direction
        target = target_intersections[trajectory][direction]
        errors[trajectory][direction]={}
        errors[trajectory][direction][threshold]={}
        errors[trajectory][direction][threshold]['incremental_weighted_update']={}
        errors[trajectory][direction][threshold]['incremental_static_update']={}
        errors[trajectory][direction][threshold]['No_update']={}

        # ------------------------------------ data processing ---------------------------------------- #
        data = merge_trejectory_data(results, trajectory, direction)# get raw data of the current trajectory and direction
        ae_score = correlated_results[key] # AE scores of the current trajectory and direction
        data_size = data.shape[0]
        model_start_time  = data.index[0]
        model_end_time = data.index[-1]
        baseline_time = data.iloc[int(data_size*base_train_portion)].name
        incremental_time = baseline_time
        base_ae = ae_score[ae_score.index<=baseline_time]
        stream_ae = ae_score[ae_score.index>baseline_time]
        base_data = data[data.index<=baseline_time]
        stream_data = data[data.index>baseline_time]
        number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
        if number_of_cols==0: # if threshold is 0, then use the target intersection only
            number_of_cols=1

        # ------------------------------------ training base model ---------------------------------------- #
        print(f"training from {model_start_time} to {baseline_time}")
        exp_name = f'univariate_{outlier_model_name}_real_time_base_model.pt'
        results_save_path = os.path.join(base_result_path, exp_name)
        results_save_weighted_path  = results_save_path
        base_model_path = results_save_path
        results_load_path = "None"
        results_load_weighted_path = "None"
        fixed_base_corr_df = "None"
        model_results, fixed_base_corr_df = test_then_train_weighted_model(base_data, base_ae, number_of_cols, target, results_load_path, base_model_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=False, static = False, base=False)
        errors[trajectory][direction][threshold]['base_model'] = model_results

        # # ------------------------------------ incremental training ---------------------------------------- #
        incremental_time = incremental_time + pd.DateOffset(hours=time_window)
        while(incremental_time<model_end_time):
            print(f"\n\ntraining from {baseline_time} to {incremental_time}")
            incremental_ae = stream_ae[(stream_ae.index>baseline_time) & (stream_ae.index<=incremental_time)]
            incremental_data = stream_data[(stream_data.index>baseline_time) & (stream_data.index<=incremental_time)]
            exp_name = f'univariate_real_time_{outlier_model_name}_incremental_static_{incremental_time}.pkl'
            exp_name_weighted = f'univariate_real_time_{outlier_model_name}_incremental_weighted_{incremental_time}.pkl'
            results_load_path = results_save_path
            results_save_path = os.path.join(base_result_path, exp_name)
            results_load_weighted_path = results_save_weighted_path
            results_save_weighted_path = os.path.join(base_result_path, exp_name_weighted)

            # ------------------------------------ training incremental models ---------------------------------------- #
            print("training incremental weighted update model")
            model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, results_load_weighted_path, results_save_weighted_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = False, base=False)
            errors[trajectory][direction][threshold]['incremental_weighted_update'][incremental_time] = model_results

            print("training incremental static update model")
            model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = True, base=False)
            errors[trajectory][direction][threshold]['incremental_static_update'][incremental_time] = model_results

            print("training No update model")
            model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, base_model_path, results_save_weighted_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = True, base=True)
            errors[trajectory][direction][threshold]['No_update'][incremental_time] = model_results


            baseline_time = incremental_time # update baseline time
            incremental_time = incremental_time + pd.DateOffset(hours=time_window) # update incremental time



 
 ------------------------ Starting trajectory T1, direction North and threshold: 0.05 ------------------------
training from 2018-01-01 00:00:00 to 2019-02-16 00:00:00
correlated intersections: ['K504']
RMSE: 47.17308088885943
Training time: 579.5522727966309


training from 2019-02-16 00:00:00 to 2019-02-16 03:00:00
training incremental weighted update model
correlated intersections: ['K504']
RMSE: 45.75887625214779
Training time: 0.1323239803314209
training incremental static update model
correlated intersections: ['K504']
RMSE: 45.75887625214779
Training time: 0.13474512100219727
training No update model
correlated intersections: ['K504']
RMSE: 45.75887625214779


training from 2019-02-16 03:00:00 to 2019-02-16 06:00:00
training incremental weighted update model
correlated intersections: ['K504']
RMSE: 12.326691634828716
Training time: 0.12235403060913086
training incremental static update model
correlated intersections: ['K504']
RMSE: 12.326691634828716
Training time: 0.12459921

ValueError: Found array with 0 sample(s) (shape=(0, 12)) while a minimum of 1 is required by StandardScaler.

In [ ]:
# ------------------------------------ save results ---------------------------------------- #
with open(os.path.join(base_result_path, f'univariate_real_time_{outlier_model_name}_threshold_{threshold}_{time_window}H_{target}.pkl'), 'wb') as f:
    pickle.dump(errors, f)

#### ---------------------------- METR-LA data processing ---------------------------- 

In [77]:
# results save path
outlier_model_name = 'PW-AE'
data_name = 'METR-LA'
base_result_path = f'../results/{data_name}/real_time_modeling'

In [78]:
with open(f'../data/{data_name}/processed/featured_fpds_raw.pickle', 'rb') as f:
    results = pickle.load(f)

In [79]:
# load data of correlated results from pickle file
with open(f'../results/{data_name}/outlier_scores/{outlier_model_name}/correlated_results.pickle', 'rb') as f:
    correlated_results = pickle.load(f)

In [80]:
thresholds = [0,0.05,0.1,0.25,0.5, 1] # thresholds for the percentage of correlated columns to keep
target_intersections = list(np.random.choice(list(results['raw'].keys()), 5, replace=False)) # select 10 intersections randomly from the 207 intersections
# declare variables
threshold = 0.05
time_window = 3
epoch_train = 100
epoch_retrain = 20
batch_size = 64
learning_rate = 0.1
hidden_size = 32
num_layers = 1
dropout = 0.2
sequence_length = 12
output_pred = 1 # number of time steps to predict
base_train_portion = 0.5 # percentage of data to use for training
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

In [ ]:
# ------------------------------------ for metr-la dataset ---------------------------------------- #
errors={}
for target in target_intersections:
    print(f"\n \n ------------------------ Starting target: {target} and threshold: {threshold} ------------------------")
    errors[target]={}
    errors[target][threshold]={}
    errors[target][threshold]['incremental_weighted_update']={}
    errors[target][threshold]['incremental_static_update']={}
    errors[target][threshold]['No_update']={}

    # ------------------------------------ data processing for base model ---------------------------------------- #
    data = merge_trejectory_data_metr(results)# get raw data of the current trajectory and direction
    ae_score = correlated_results['df'] # AE scores of the current trajectory and direction
    data_size = data.shape[0]
    model_start_time  = data.index[0]
    model_end_time = data.index[-1]
    baseline_time = data.iloc[int(data_size*base_train_portion)].name
    incremental_time = baseline_time
    base_ae = ae_score[ae_score.index<=baseline_time]
    stream_ae = ae_score[ae_score.index>baseline_time]
    base_data = data[data.index<=baseline_time]
    stream_data = data[data.index>baseline_time]
    number_of_cols = math.ceil(len(ae_score.columns)*threshold) # number of outlier weighted intersections
    if number_of_cols==0: # if threshold is 0, then use the target intersection only
        number_of_cols=1

    # ------------------------------------ training base model ---------------------------------------- #
    print(f"training from {model_start_time} to {baseline_time}")
    exp_name = f'univariate_{outlier_model_name}_real_time_base_model.pt'
    results_save_path = os.path.join(base_result_path, exp_name)
    results_save_weighted_path  = results_save_path
    base_model_path = results_save_path
    results_load_path = "None"
    results_load_weighted_path = "None"
    fixed_base_corr_df = "None"
    model_results, fixed_base_corr_df = test_then_train_weighted_model(base_data, base_ae, number_of_cols, target, results_load_path, base_model_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=False, static = False, base=False)
    errors[target][threshold]['base_model'] = model_results

    # # ------------------------------------ incremental training ---------------------------------------- #
    incremental_time = incremental_time + pd.DateOffset(hours=time_window)
    while(incremental_time<model_end_time):
        print(f"\n\ntraining from {baseline_time} to {incremental_time}")
        incremental_ae = stream_ae[(stream_ae.index>baseline_time) & (stream_ae.index<=incremental_time)]
        incremental_data = stream_data[(stream_data.index>baseline_time) & (stream_data.index<=incremental_time)]
        exp_name = f'univariate_real_time_{outlier_model_name}_incremental_static_{incremental_time}.pkl'
        exp_name_weighted = f'univariate_real_time_{outlier_model_name}_incremental_weighted_{incremental_time}.pkl'
        results_load_path = results_save_path
        results_save_path = os.path.join(base_result_path, exp_name)
        results_load_weighted_path = results_save_weighted_path
        results_save_weighted_path = os.path.join(base_result_path, exp_name_weighted)

        # ------------------------------------ training incremental models ---------------------------------------- #
        print("training incremental weighted update model")
        model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, results_load_weighted_path, results_save_weighted_path, hidden_size, output_pred, num_layers, dropout, device,sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = False, base=False)
        errors[target][threshold]['incremental_weighted_update'][incremental_time] = model_results

        print("training incremental static update model")
        model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, results_load_path, results_save_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = True, base=False)
        errors[target][threshold]['incremental_static_update'][incremental_time] = model_results

        print("training No update model")
        model_results,top_corr_df = test_then_train_weighted_model(incremental_data, incremental_ae, number_of_cols, target, base_model_path, results_save_weighted_path, hidden_size, output_pred, num_layers, dropout, device, sequence_length, epoch_train, epoch_retrain, fixed_base_corr_df, pre_trained=True, static = True, base=True)
        errors[target][threshold]['No_update'][incremental_time] = model_results


        baseline_time = incremental_time # update baseline time
        incremental_time = incremental_time + pd.DateOffset(hours=time_window) # update incremental time


In [75]:
# ------------------------------------ save results ---------------------------------------- #
with open(os.path.join(base_result_path, f'univariate_real_time_{outlier_model_name}_threshold_{threshold}_{time_window}H_{target}.pkl'), 'wb') as f:
    pickle.dump(errors, f)

# EXTRA

In [ ]:
# def test_then_train_model(data, ae_score, number_of_cols, target, results_load_path, results_save_path, base_mode_path, pre_trained=True) :
#     result_dict = {}

#     # ------------------------------------ data processing ---------------------------------------- #
#     top_corr_df = ae_score.corr()[target].sort_values(ascending=False)[:number_of_cols] # get the top correlated intersections
#     isct_inc = top_corr_df.index.tolist()
#     print("correlated intersections: {}".format(isct_inc))
#     df = data[isct_inc].copy(deep=True)
#     # df = df.mul(top_corr_df, axis=1)
#     df = df.astype('float32')
#     sequence_length = 12 # number of time steps to look back
#     n_features = len(isct_inc) # number of features (correlated intersections)
#     output_pred = 1 # number of time steps to predict
#     n_obs = sequence_length * n_features # number of columns in the input
#     reframed = series_to_supervised(df, sequence_length, output_pred)
#     train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


# # # # ------------------------------------ modelling ---------------------------------------------- #
#     # define model, loss function and optimizer
#     model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
#     model = model.to(device)
#     loss_fn = torch.nn.MSELoss()
#     optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
#     if pre_trained==True:
#         model, optimiser = load_ckp(results_load_path, model, optimiser)
    

#     # ------------------------------------ testing ---------------------------------------------- #
#     yhat = model_evaluation( model, train_X , device)
#     result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
#     result_dict['MAE'] = mean_absolute_error(yhat,train_y)
#     result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
#     print("RMSE: {}".format(result_dict['RMSE']))


#    # ------------------------------------ training ---------------------------------------------- #
#     start = time.time()
#     if pre_trained==False:
#         history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=50)
#     else:
#         history, model = train_model(model, train_X,train_y, loss_fn, optimiser, device, epochs=1)
#     end = time.time()
#     checkpoint = {'state_dict': model.state_dict(),'optimizer': optimiser.state_dict()}
#     torch.save(checkpoint, results_save_path)
#     print("Training time: {}".format(end-start))
#     result_dict['history'] = history
#     result_dict['intersedctions'] = isct_inc
#     result_dict['train_time'] = end-start

    
#     return result_dict, top_corr_df

In [25]:
# def test_base_model(data, ae_score, number_of_cols, target, results_load_path, top_corr_df,pre_trained=True):
#     result_dict = {}

#     # ------------------------------------ data processing ---------------------------------------- #
#     isct_inc = top_corr_df.index.tolist()
#     print("correlated intersections: {}".format(isct_inc))
#     df = data[isct_inc].copy(deep=True)
#     df = df.mul(top_corr_df, axis=1)
#     df = df.astype('float32')
#     sequence_length = 12 # number of time steps to look back
#     n_features = len(isct_inc) # number of features (correlated intersections)
#     output_pred = 1 # number of time steps to predict
#     n_obs = sequence_length * n_features # number of columns in the input
#     reframed = series_to_supervised(df, sequence_length, output_pred)
#     train_X, train_y, scl = preprocess_df(reframed, n_obs, n_features, sequence_length)


# # # # ------------------------------------ modelling ---------------------------------------------- #
#     # define model, loss function and optimizer
#     model = LSTM_uni(input_dim = n_features, hidden_dim = 32, layer_dim = 1, output_dim = output_pred, dropout_prob= 0.2)
#     model = model.to(device)
#     loss_fn = torch.nn.MSELoss()
#     optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
#     if pre_trained==True:
#         model, optimiser = load_ckp(results_load_path, model, optimiser)
    

#     # ------------------------------------ testing ---------------------------------------------- #
#     yhat = model_evaluation( model, train_X , device)
#     result_dict['RMSE'] = sqrt(mean_squared_error(yhat,train_y))
#     result_dict['MAE'] = mean_absolute_error(yhat,train_y)
#     result_dict['df'] = pd.DataFrame({"Real":train_y,"Predicted":yhat})
#     result_dict['intersedctions'] = isct_inc
#     print("RMSE: {}".format(result_dict['RMSE']))
    
#     return result_dict, top_corr_df